## Test the /search endpoints for auxip and cadip

In [ ]:
# Init environment before running a demo notebook.
from resources.utils import *
init_demo()
from resources.utils import * # reload the global vars again

In [ ]:
from dataclasses import dataclass
from datetime import datetime
import itertools
import json

@dataclass
class Error:
    """Errors in this tests."""
    endpoint: str
    params: dict
    messages: list[str]
errors: list[Error] = []

def search_and_check(endpoint, **kwargs):
    """Call the /search endpoint, check the results, return features."""

    # Build the query parameters
    params = {"query": {}}
    for property, value in kwargs.items():

        if property in ("limit", "ids"):
            params[property] = value

        # Date interval from the first and last date
        elif property == "datetime":
            params[property] = f"{value[0]}/{value[-1]}".replace("+00:00", "Z")
            expected_dtmin = datetime.fromisoformat(value[0])
            expected_dtmax = datetime.fromisoformat(value[-1])

        # query parameters
        else:
            params["query"][property] = {"eq": value}

    print(f"Call POST {endpoint!r} with params: {list(kwargs.keys())}")

    # Call the search endpoint, read the returned features
    response = http_session.post(endpoint, json=params)

    if response.status_code != 200:
        errors.append(Error(endpoint, params, [f"Status code: {response.status_code}\n{response.content}"]))
        return

    features = response.json()["features"]
    if not features:
        errors.append(Error(endpoint, params, [f"No features returned"]))
        return

    # Check errors
    messages: list[str] = []
    for property, expected_value in kwargs.items():

        if property == "limit":
            feature_count = len(features)
            # TO BE FIXED BY https://pforge-exchange2.astrium.eads.net/jira/browse/RSPY-131 ?
            if feature_count > expected_value:
                messages.append(f"{feature_count} features returned, expected max: {expected_value}")
            continue

        # Check that each feature property value is in the requested range
        for f in features:
            if property == "datetime":
                feature_dt = datetime.fromisoformat(f["properties"][property])
                if (feature_dt < expected_dtmin) or (feature_dt > expected_dtmax):
                    messages.append(
                        f"{feature_dt.isoformat()!r} should be between {expected_dtmin.isoformat()!r} "
                        f"and {expected_dtmax.isoformat()!r}")
            else:
                if property == "ids":
                    feature_param = f["id"]
                else:
                    feature_param = f["properties"][property]
                if feature_param not in expected_value:
                    messages.append(f"{feature_param!r} should be in {expected_value}")

    if messages:
        errors.append(Error(endpoint, params, messages))
    return features

for service in "auxip", "cadip":

    # For better readability
    auxip = (service == "auxip")
    cadip = (service == "cadip")

    if auxip:
        endpoint = f"{auxip_client.href_adgs}/auxip/search"
    elif cadip:
        endpoint = f"{cadip_client.href_cadip}/cadip/search"

    # Get all auxip products or cadip sessions
    features = search_and_check(endpoint, limit=1000)

    # Read all possible properties
    properties = {
        "limit": 1,
        "ids": list({f["id"] for f in features}),
        "datetime": sorted({f["properties"]["datetime"] for f in features}),
        "platform": list({f["properties"]["platform"] for f in features}),
    }
    if auxip:
        properties.update({
            "constellation": list({f["properties"]["constellation"] for f in features}),
            "product:type": list({f["properties"]["product:type"] for f in features})
        })

    # Keep 1 or 2 elements for each propertiy
    for key, value in properties.items():
        if isinstance(value, (int, str)): # only for lists
            continue
        if len(value) == 0: # if 0 element: do nothing
            continue        
        if len(value) > 1: # if at least 2 elements: remove the first one
            value.pop(0) 
        # Keep 1 or 2 elements
        properties[key] = value[:2] if key in ("ids", "datetime") else value[0]

    # Test all combinations of n properties
    for length in range(1, len(properties)+1):
        for keys in itertools.combinations(properties.keys(), length):
            search_and_check(endpoint, **{key: properties[key] for key in keys})

if errors:
    message = "\n## Error message start ##\n"
    for error in errors:
        sep = "\n  - "
        message += (            
            f"\nPOST {error.endpoint!r}\n{json.dumps(error.params, indent=2)}\n"
            f"Error(s):{sep}{sep.join(error.messages)}\n")
    message += "\n## Error message finish ##\n"
    raise RuntimeError(message)